# Goal: Do both Data cleaning & Feature Extraction

In [15]:
import polars as pl
import seaborn as sns

In [16]:
path = r"C:\Users\Rudra\Desktop\rural-financial-inclusion-govt-scheme-recommendation\parquet-data\lev-06\data\lev-06_merged.parquet"

pdf = pl.read_parquet(path)
pdf.collect_schema()

Schema([('Survey_Name', String),
        ('Year', String),
        ('FSU_Serial_No', String),
        ('Sector', String),
        ('State', String),
        ('NSS_Region', String),
        ('District', String),
        ('Stratum', String),
        ('Sub_stratum', String),
        ('Panel', String),
        ('Sub_sample', String),
        ('FOD_Sub_Region', String),
        ('Sample_SU_No', String),
        ('Sample_Sub_Division_No', String),
        ('Second_Stage_Stratum_No', String),
        ('Sample_Household_No', String),
        ('Questionnaire_No', String),
        ('Level', String),
        ('Item_Code', String),
        ('Total_Consumption_Quantity', String),
        ('Total_Consumption_Value', String),
        ('Source', String),
        ('Multiplier', Int64)])

In [17]:
pdf = pdf.filter(
    pl.col('State') == '23'
)

pdf['State'].unique()

State
str
"""23"""


In [18]:
lev_06 = [
    'Total_Consumption_Quantity',
    'Total_Consumption_Value',
    'Source',
    'Multiplier',
]

pdf = pdf[lev_06]

In [19]:
pdf

Total_Consumption_Quantity,Total_Consumption_Value,Source,Multiplier
str,str,str,i64
"""""","""30""","""1""",58558
"""0.100""","""20""","""1""",58558
"""""","""60""","""1""",58558
"""""","""10""","""1""",58558
"""""","""50""","""1""",58558
…,…,…,…
"""""","""20""","""""",102702
"""""","""95""","""""",102702
"""""","""30""","""""",102702


In [20]:
pdf = pdf.with_columns(
    [pl.col(col).cast(pl.Int32, strict=False) for col in lev_06]
)
pdf.schema

Schema([('Total_Consumption_Quantity', Int32),
        ('Total_Consumption_Value', Int32),
        ('Source', Int32),
        ('Multiplier', Int32)])

In [21]:
pdf.null_count()

Total_Consumption_Quantity,Total_Consumption_Value,Source,Multiplier
u32,u32,u32,u32
188044,0,58004,0


In [22]:
pdf.null_count() / pdf.shape[0]

Total_Consumption_Quantity,Total_Consumption_Value,Source,Multiplier
f64,f64,f64,f64
0.972185,0.0,0.29988,0.0


In [23]:
def check_unique(col):
    print(f"{col} contains : {pdf[col].n_unique()}")
    display(pdf[col].unique())
    display(f" This {col} have the skewness {pdf[col].skew()}")
    print(f"{'='*50}")

In [24]:
def plot_hist(col):
    sns.histplot(pdf[col])

In [25]:
# plot_hist('OutOfHome_Consumption_Quantity')

In [26]:
for col in pdf.columns:
    check_unique(col)

Total_Consumption_Quantity contains : 59


Total_Consumption_Quantity
i32
null
1
2
3
4
…
200
220
250


' This Total_Consumption_Quantity have the skewness 3.264744207030345'

Total_Consumption_Value contains : 734


Total_Consumption_Value
i32
1
2
3
4
5
…
4400
4600
4700


...

Source
i32
null
1
5
6
9


' This Source have the skewness 17.561301500294718'

Multiplier contains : 1421


Multiplier
i32
1913
1983
2250
2400
3000
…
613888
653383
828400


' This Multiplier have the skewness 4.048097778925166'

In [27]:
pdf.columns

['Total_Consumption_Quantity',
 'Total_Consumption_Value',
 'Source',
 'Multiplier']

In [28]:
for col in pdf.columns:
    pdf = pdf.with_columns(
        pl.col(col).fill_null(strategy='forward')
    )

In [33]:
pdf = pdf.with_columns(
    pl.col('Total_Consumption_Quantity').fill_null(strategy='mean')
)

In [34]:
pdf.null_count()

Total_Consumption_Quantity,Total_Consumption_Value,Source,Multiplier
u32,u32,u32,u32
0,0,0,0


In [ ]:
e
pdf.write_csv(r"C:\Users\Rudra\Desktop\rural-financial-inclusion-govt-scheme-recommendation\parquet-data\lev-05\data2\lev_05_mp_clean.csv")
pdf.write_parquet(r"C:\Users\Rudra\Desktop\rural-financial-inclusion-govt-scheme-recommendation\parquet-data\lev-05\data2\lev_05_mp_clean.parquet", compression="zstd")
print('Saved 🙌')

Saved 🙌
